In [1]:
import sqlalchemy as db

In [2]:
# Path to database
db_file = "C:/users/jerry/Desktop/RestoredToEden.db"

# Set up SQLAlchemy connection
engine = db.create_engine('sqlite:///' + db_file)
cnxn = engine.connect()

metadata = db.MetaData()
metadata.reflect(cnxn)

InformationSheet = metadata.tables['InformationSheet']

In [3]:
def get_table_contents(table):
    """ inputs: table containing items to query
        returns: dict of {col headings: col values}
    """
    query = db.select([table])
    headings = cnxn.execute(query).keys()
    query = db.select([table])
    values = cnxn.execute(query).fetchone()

    contents = {}
    # Create dict struct
    for i in range(len(headings)):
        contents[headings[i]] = values[i]

    return contents

get_table_contents(InformationSheet).values()

dict_values(['Simple instructions can be found on each product. [customer name], [detailed description inserted here from file]', 'It is recommended that you strip back your skincare routine to the basics and start with a blank slate. It is also recommended that you do not use other new skincare products while you are testing our products on your skin. The reason for this is you will not really know which of the products caused a reaction if you introduce too many at one time. If you usually do not have a problem with reactions on your skin, you should not have a reaction.', 'Store below 25 degrees celsius. Keep out of direct sunlight. Store in a dry place. Do not allow water to enter your product.', 'dwwd', 'We go to great lengths to hand select every ingredient. Please do not hesitate to let us know if you are not having the desired effect after using this product for the first 6 weeks. We will ensure that we analyse your needs further and make the necessary adjustments to your formu

In [4]:
def get_col_names(table):
    query = db.select([table])
    return cnxn.execute(query).keys()

get_col_names(InformationSheet)

['Recommendations For Use',
 'For Best Results',
 'Care & Storage Instructions',
 'Directions & Warnings Before Use',
 'Our promises to you',
 'Used By & Best Before Date']

In [5]:
def insert_query(table, data):
    """ data: dictionary of column name and its value
    """
    query = table.insert().values(data)
    cnxn.execute(query)
    
def update_query(table, data):
    query = db.update(table).values(data)
    cnxn.execute(query)
    

In [6]:
data = {"Recommendations For Use": "Simple instructions can be found on each product.  [customer name], [detailed description inserted here from file]",
        "For Best Results": "It is recommended that you strip back your skincare routine to the basics and start with a blank slate.  It is also recommended that you do not use other new skincare products while you are testing our products on your skin.  The reason for this is you will not really know which of the products caused a reaction if you introduce too many at one time.  If you usually do not have a problem with reactions on your skin, you should not have a reaction.",
        "Care & Storage Instructions": "Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.",
        "Directions & Warnings Before Use": "Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.",
        "Our promises to you": "We go to great lengths to hand select every ingredient.  Please do not hesitate to let us know if you are not having the desired effect after using this product for the first 6 weeks.  We will ensure that we analyse your needs further and make the necessary adjustments to your formulation.\nWe select only natural and organic ingredients in order to formulate products that are not only gentle on your skin but are also effective.  Each of our ingredients goes through a selection process to ensure the highest quality.\nPlease do not hesitate to contact us if you have any questions or concerns about any of our products.  We would be happy to help.",
        "Used By & Best Before Date": "This blend must be used within …. months of blending.\nDate Blended: [date blended]\nBest Before Date: [best before date]"
       }

update_query(InformationSheet, data)

In [7]:
query = db.select([InformationSheet])
paragraphs = cnxn.execute(query).fetchone()
col_names = get_col_names(InformationSheet)

paragraphs[-1]

'This blend must be used within …. months of blending.\nDate Blended: [date blended]\nBest Before Date: [best before date]'

In [9]:
def add_column(table_name, column):
    column_name = column.compile(dialect=engine.dialect)
    column_type = column.type.compile(engine.dialect)
    print(str(column_name))
    print(column_type)
    cnxn.execute('ALTER TABLE %s ADD COLUMN %s %s' % (table_name, column_name, column_type))

column = db.Column('new_column_name', db.Text)
add_column("InformationSheet", column)

new_column_name
TEXT


OperationalError: (sqlite3.OperationalError) duplicate column name: new_column_name
[SQL: ALTER TABLE InformationSheet ADD COLUMN new_column_name TEXT]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [72]:
import copy
# Delete column
metadata = db.MetaData()
metadata.reflect(cnxn)
InformationSheet = metadata.tables['InformationSheet']

# Get columns to keep
col_drop = ["new_column_name", "Field8"]
new_cols = []
for column in InformationSheet.columns:
    if not (column.name in col_drop):
        column_copy = copy.deepcopy(column)
        column_copy.table = None
        new_cols.append(column_copy)

# Save data to keep
new_content = get_table_contents(InformationSheet)
del new_content["Field8"]
del new_content["new_column_name"]

#new_table = db.Table("copy", metadata, *(column for column in new_cols))

In [73]:
new_content.items()

dict_items([('Recommendations For Use', 'Simple instructions can be found on each product.  [customer name], [detailed description inserted here from file]'), ('For Best Results', 'It is recommended that you strip back your skincare routine to the basics and start with a blank slate.  It is also recommended that you do not use other new skincare products while you are testing our products on your skin.  The reason for this is you will not really know which of the products caused a reaction if you introduce too many at one time.  If you usually do not have a problem with reactions on your skin, you should not have a reaction.'), ('Care & Storage Instructions', 'Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.'), ('Directions & Warnings Before Use', 'Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.'), ('Our promises to you', 'We go 

In [64]:
metadata.create_all(engine)

In [68]:
# Add data to new table
insert_query(metadata.tables['copy'], get_table_contents(InformationSheet))

CompileError: Unconsumed column names: new_column_name, Field8

In [ ]:
def insert_query(table, data):
    """ data: dictionary of column name and its value
    """
    query = table.insert().values(data)
    cnxn.execute(query)

In [11]:
copy = metadata.tables["copy"]
copy.drop(engine)

In [31]:
def setup_df(csv_data):
    """ Takes csv reader data and converts it to dataframe. """
    df = pd.DataFrame(csv_data)
    # Set row 0 as column names
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    # Reindex
    df.reset_index(inplace=True)
    return df

In [34]:
import csv
import pandas as pd

# Customer orders
data = csv.reader(open("C:/users/jerry/Desktop/Restored To Eden/orders.csv"))
orders_df = setup_df(data)

# Product catalog
data = csv.reader(open("C:/users/jerry/Desktop/Restored To Eden/catalog_products.csv"))
catalog_df = setup_df(data)
df_prods = pd.DataFrame(catalog)

NameError: name 'catalog' is not defined

In [107]:
def get_product_type(product_name):
    # Cross query product catalog
    types = catalog_df[catalog_df["additionalDescription1"].str.constains(product_name)].additionalInfoDescription1.iloc[0]

In [104]:
type(catalog_df["additionalInfoDescription1"])

pandas.core.series.Series

In [115]:
type(catalog_df[catalog_df["additionalInfoDescription1"].str.contains("Serum")].additionalInfoDescription1)

pandas.core.series.Series

In [117]:
catalog_df[catalog_df["additionalInfoDescription1"].str.contains("Serum")].additionalInfoDescription1.iloc[0]

'<p>Hydration Serum, Oil Serum</p>\n'

In [57]:
templates = ["2 Birds Serum", "Cleansing Lotion", "Day Cream", "Man Cream", 
             "Hydration Serum", "Night Cream", "Oil Serum", "Oil Cleanser", 
             "Toner"]

# Go through orders and find the production template to use
i=0
for item in orders_df[["Item's Name"]].iloc[:,0]:
    # Find product types
    
    
    # Go through formulation worksheets
    for sheet in templates:
        if item in sheet:
            print(sheet)
            i+=1
            break

Hydration Serum
Oil Serum
Oil Serum
Oil Serum
Night Cream
Day Cream
Hydration Serum
Oil Serum
Night Cream
Hydration Serum
Night Cream
Oil Cleanser
Night Cream
Oil Cleanser
Cleansing Lotion
Toner
Day Cream
Oil Cleanser
Cleansing Lotion
Toner
Oil Serum
Hydration Serum


# Section allows editing of information sheet
Store info in an excel sheet

In [2]:
from openpyxl import load_workbook
from datetime import *
import pandas as pd
import os

In [282]:
class ButtonWrapper:
    def __init__(self, button, label, text_box):
        self.df = None
        self.button = button
        self.label = label
        self.text_box = text_box
        
        self.button.clicked.connect(self.del_section)
        
    def del_section(self):
        df = pd.DataFrame({self.label.text(): [self.text_box.toPlainText()]})
        self.label.deleteLater()
        self.text_box.deleteLater()
        self.button.deleteLater()
        
        self.df = df

In [284]:
import sys
from PySide2.QtWidgets import *

paragraphs_df = pd.read_excel(info_sheet_path)

class Form(QDialog):
    def __init__(self, contents, parent=None):
        super(Form, self).__init__(parent)
        
        self.paragraphs_df = contents
        
        self.setWindowTitle("Information Sheet Editor")
        self.resize(700, 500)
        self.layout = QGridLayout()

        # Save button
        button = QPushButton("Save")
        button.clicked.connect(self.refresh)
        self.layout.addWidget(button, 0, 0)

        # Add new section button
        button = QPushButton("Add section")
        button.clicked.connect(self.add_section)
        self.layout.addWidget(button, 1, 0)
        
        self.del_buttons = []
        for i in range(contents.shape[1]):
            # Create label
            l = QLabel(list(contents)[i])
            self.layout.addWidget(l, 2*i+2, 0)
            
            # Create text boxes
            t = QTextEdit(contents.iloc[0,i])
            self.layout.addWidget(t, 2*i +2 + 1, 0)
            
            button = QPushButton("del")
            self.del_buttons.append(ButtonWrapper(button, l, t))
            self.layout.addWidget(button, 2*i +2 + 1, 1)

#         # Delete section button
#         button = QPushButton("Delete section")
#         button.clicked.connect(self.del_section)
#         self.layout.addWidget(button)

        # Set dialog layout
        self.setLayout(self.layout)

    def add_section(self):
        dialog = QDialog()
        dialog.setWindowTitle("New Section")
        layout = QVBoxLayout()

        # New section heading
        l = QLabel("Section Title")
        layout.addWidget(l)
        e = QLineEdit()
        layout.addWidget(e)

        # New section content
        l_content = QLabel("Section Content")
        layout.addWidget(l_content)
        e_content = QTextEdit()
        layout.addWidget(e_content)

        # Button adds new info to SQLite
        button = QPushButton("Ok!")
        button.clicked.connect(lambda: self.add_N_close(dialog, e.text(), e_content.toPlainText()))

        dialog.setLayout(layout)
        dialog.exec_() 

    def add_N_close(self, dialog, n_heading, n_body):
        new_section_df = pd.DataFrame({n_heading: [n_body]})
        self.paragraphs_df = pd.concat([paragraphs_df, new_section_df], axis=1)
        
        l = QLabel(n_heading)
        t = QTextEdit(n_body)
        self.text_boxes.append(t)
        
        # Insert new section before the buttons
        print(self.layout.count())
        pos = self.layout.rowCount()
        self.layout.addWidget(l, pos, 0)
        self.layout.addWidget(t, pos+1, 0)

        dialog.close()
    
     
    def refresh(self):
        for b_wrap in self.del_buttons:
            if b_wrap.df != None:
                print(b_wrap.df)
        # Export paragraph dfs to xlsx to load later
        #self.paragraphs_df.to_excel("test.xlsx")


In [280]:
# Read current info sheet
working_dir = os.getcwd()
info_sheet_path = working_dir + r"\BespokeAutoSystem\InformationSheet.xlsx"
paragraphs_df = pd.read_excel(info_sheet_path)
paragraphs_df

,Recommendations For Use,For Best Results,Care & Storage Instructions,Directions & Warnings Before Use,Our promises to you,Used By & Best Before Date
0,eek,Saradomin,Zaros,Zamorak,Bandos,Armadyl


In [57]:
# Create the Qt Application
app = QApplication(sys.argv)


RuntimeError: Please destroy the QApplication singleton before creating a new QApplication instance.

In [286]:
# Create and show the form
form = Form(paragraphs_df)
form.show()
# Run the main Qt loop
sys.exit(app.exec_())

AttributeError: 'ButtonWrapper' object has no attribute 'df'

SystemExit: 0

# This Section opens excel worksheet template and writes to it!
1. Read input:
    - Ingredient list
    - Product name
    - Product type
    - Customer name
2. Open corresponding excel template to product type
3. Fill out header information
4. Go through ingredients and fill in main table entries
5. Run calc. function to fill grams based on product formulation
6. Output .xlsx

In [1]:
from openpyxl import load_workbook
from datetime import *
import pandas as pd

In [3]:
## Read input
class FormInfo:
    def __init__(self, ingredients, prod_name, prod_type, client_name):
        self.ingredients = ingredients
        self.prod_name = prod_name
        self.prod_type = prod_type
        self.client_name = client_name

form = FormInfo("Horse Chestnut Extract, Kaolin Clay, Aloe Vera Juice, Carrot Seed Essential Oil", 
                "Night Cream", "Cleansing Lotion", "Jane Doe")

In [4]:
## Find correct template
template_path = "C:/users/jerry/Desktop/Restored To Eden/Formulation Templates/"

full_path = template_path + form.prod_type + " Worksheet.xlsx"
workbook = load_workbook(filename=full_path)
sheet = workbook.active

In [5]:
## Fill out header info
# Name
sheet["B1"] = form.client_name
# Product Type
sheet["B2"] = form.prod_type
# Product Formulation
sheet["B5"] = 100

In [6]:
## Fill out main table entries
ingredients_df = pd.read_csv("C:/users/jerry/Desktop/Restored To Eden/RTE Ingredients Database.csv", 
                               encoding = "ISO-8859-1").set_index("INGREDIENT COMMON NAME")
ingredients_df = ingredients_df.loc[~ingredients_df.index.duplicated(keep="first")]
ingredients_df

,INGREDIENT INCI NAME,In Stock,Quantity in Stock (in grams),Supplier Name,CAS NO.,NICNAS Check Completed,Product Type,Skin Type,TYPE OF INGREDIENT,ESSENTIAL OIL NOTE,...,VISCOSITY,ABSORPTION RATE,COMEDOGENIC RATING,Composition,Recommended Concentration,Allergens,Relevant References,Other Important Information,4,K
INGREDIENT COMMON NAME,,,,,,,,,,,,,,,,,,,,,
Acai Oil Certified Organic,Euterpe oleracea (Acai) oil (certified organic),Yes,NaN,AWO,NaN,No,Anhydrous,"Dry Skin,Sensitive & Eczema Prone Skin,Mature ...","Carrier Oil,anhydrous base",NaN,...,Light,Fast,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aloe Vera Juice,Aloe Barbadensis (Aloe Vera) juice,Yes,NaN,ND,NaN,Yes,NaN,Mature & Aging Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aloe Vera powder and purified Aqua - and prese...,NaN,NaN,NaN
Alpinia Leaf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mature & Aging Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alpinia Root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mature & Aging Skin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricot Kernel Oil,Prunus armeniaca (Apricot) kernel oil (certifi...,Yes,NaN,AWO,NaN,Yes,"Anhydrous,Emulsion","Dry Skin,Sensitive & Eczema Prone Skin,Mature ...","Carrier Oil,anhydrous base",NaN,...,Light,Fast,2.0,NaN,NaN,NaN,Review Article\nPolyphenols as active ingredie...,Thinner consistency\nCleansing properties\n\nA...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin E Natural (Mixed tocopherols 70%),Mixed tocopherols 70% (Vitamin E natural),Yes,NaN,AWO,7695-91-2,No,"Anhydrous,Emulsion",Mature & Aging Skin,"Antioxidant,anhydrous high performance",NaN,...,NaN,NaN,NaN,NaN,0.5%-2%,NaN,NaN,"Components: Mixed Tocopherols (d-alpha-, d-bet...",NaN,NaN
Walnut Oil,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,"Carrier Oil,anhydrous base",NaN,...,NaN,Medium,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
White Willow Bark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive & Eczema Prone Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
ingredients_df.loc["Aloe Vera Juice"]["SKIN PROBLEM"]

'Inflamed,Irritated,Fine lines,Wrinkles,Predisposed to acne'

In [13]:
sheet[f"E{6}"].value.lower()

'total grams/\namount in grams'

In [12]:
# get list of ingredients
ingredients = form.ingredients.split(",")

realloc = 0
cell_dict = {}
# Fill main table entries
i = 0  # ingredient index
j = 7  # cell index
while i < len(ingredients):
    
    # Stop loop if reached end of template
    if type(sheet[f"A{j}"]) != type(0.0):
        break
        
    ingredient_name = ingredients[i]
    
    # Fill cell if ingredient is same type as template
    if ingredients_df.loc[ingredient_name]["TYPE OF INGREDIENT"] in sheet[f"B{j}"].value:
        # Fill INCI
        sheet[f"A{j}"] = ingredients_df[["INGREDIENT INCI NAME"]]
        # Fill ingredient name
        sheet[f"B{j}"] = ingredient_name
        # Fill needs targted column if applicable
        try:
            if sheet[f"F{6}"].value.lower() == "needs targetting":
                sheet[f"F{6}"] = ingredients_df.loc[ingredient_name]["SKIN PROBLEM"]
        except:
            print("No needs targetting column.")
        
        # Store cell and value for realloc
        cell_dict[f"D{j}"] = sheet[f"D{j}"].value
        i += 1
        j += 1

    # Otherwise record missing % chunk
    else:
        print(sheet[f"D{j}"].value)
        realloc += sheet[f"D{j}"].value
        cell_dict[f"D{j}"] = 0
        j += 1

# Reallocate surplus %
if realloc > 0:
    realloc_chunks(realloc, cell_dict, sheet)

In [70]:
cell_dict = {"D7": 75.0, "D8": 5.0, "D9": 2.0, 
             "D10": 6.5, "D11": 5.0, "D12": 2.0, "D13": 0.0}
realloc = 4.5

def realloc_chunks(realloc, cell_dict, sheet):
    ## Percentage reallocator
    tot = sum(cell_dict.values())
    for i in range(len(cell_dict)):
        cur_num = list(cell_dict.values())[i]
        cell_dict[list(cell_dict.keys())[i]] = cur_num + (cur_num / tot) * realloc 

    return cell_dict

def write_weights(cell_dict, sheet): 
    del_rows = []
    # Write realloc chunks to sheet
    for cell in cell_dict.keys():
        # record row for deletion
        if (cell_dict[cell]) == 0:
            del_rows.append(cell)
        sheet[cell] = cell_dict[cell]
    
        
    # delete empty rows
    for cell in del_rows:
        sheet.delete_rows((int)(cell[1:]))
        
    return sheet
        
realloc_dict = realloc_chunks(realloc, cell_dict, sheet)
sheet = write_weights(realloc_dict, sheet)

In [71]:
## Export
save_path = "C:/users/jerry/Desktop/Restored To Eden/test forms/"
workbook.save(save_path + form.client_name + "-" + form.prod_type + ".xlsx")

In [4]:
# Upload file to google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# For listdir() function
import os

# Authenication
g_login = GoogleAuth()
# Create local webserver and auto handles authentication
g_login.LocalWebserverAuth()
drive = GoogleDrive(g_login)

# Directory of the file to upload
path = os.getcwd() + r"\test forms"

for x in os.listdir(path):
    print(type(x))
    f = drive.CreateFile({'title': x})
    f.SetContentFile(os.path.join(path, x))
    f.Upload()
    # Clear variable to prevent memory leak
    f = None

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=54206686856-fa4bd9fgt5vq81l7ipdvv8e3s9137jmq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
<class 'str'>


# Config File Testing

In [23]:
import json
# Json is a dictionary in python
Config = {"ConstraintSettings": {
    "DayCream": {"Viscosity": "Light", "Absorbency": "Fast"},
    "ManCream": {"Viscosity": "Medium", "Absorbency": "Medium"},
    "NightCream": {"Viscosity": "Heavy", "Absorbency": "Slow"},
    "OilCleanser": {"Viscosity": "Medium", "Absorbency": "Medium"}
},
          "Viscosity": ["light", "medium", "heavy"],
          "Absorbency": ["slow", "medium", "fast"],
          "ComedogenicRating": [0, 1, 2 ,3 ,4 ,5]}
# Writing config file to output
json_obj = json.dumps(Config, indent=4)
with open("config.json", "w") as outfile:
    outfile.write(json_obj)
    
# Reading config file
with open("config.json") as json_file:
    data = json.load(json_file)
data

{'ConstraintSettings': {'DayCream': {'Viscosity': 'Light',
   'Absorbency': 'Fast'},
  'ManCream': {'Viscosity': 'Medium', 'Absorbency': 'Medium'},
  'NightCream': {'Viscosity': 'Heavy', 'Absorbency': 'Slow'},
  'OilCleanser': {'Viscosity': 'Medium', 'Absorbency': 'Medium'}},
 'Viscosity': ['light', 'medium', 'heavy'],
 'Absorbency': ['slow', 'medium', 'fast'],
 'ComedogenicRating': [0, 1, 2, 3, 4, 5]}

# Find distance between 2 lines

In [25]:
# Get coordinates by coverting categorical data to numeric
# Line coords: (x=Viscosity, y=Absorbency, z=Comedogenic rating)
product_type = "OilCleanser"
# Find coordinates of points (a) for optimum
try:
    x1_str = data["ConstraintSettings"][product_type]["Viscosity"].lower()
    x1 = [option.lower() for option in data["Viscosity"]].index(x1_str)
except:
    x1 = 0

try:
    y1_str = data["ConstraintSettings"][product_type]["Absorbency"].lower()
    y1 = [option.lower() for option in data["Absorbency"]].index(y1_str)
except:
    y1 = 0

try:
    z1_str = data["ConstraintSettings"][product_type]["ComedogenicRating"].lower()
    z1 = [option.lower() for option in data["ComedogenicRating"]].index(z1_str)
except:
    z1 = 0
    
a1 = (x1, y1, z1)

# a2 is the point of the selected ingredient list - take average of all ingredients?
# Since all points go through origin the directional vector is just the vector itself

(1, 1, 0)

In [36]:
import numpy as np
# distance between 2 skew lines
# Points
a1 = [0, 2, -1]
a2 = [1, 0, -1]
# Direction vectors
b1 = [1, 1, 2]
b2 = [1, 1, 3]

PQ = []
for i in range(len(a1)):
    PQ.append(a2[i] - a1[i])
    
np.linalg.norm(np.dot(PQ, np.cross(b1, b2))) / np.linalg.norm(np.cross(b2, b1))

2.1213203435596424